In [1]:
import pandas as pd 
import numpy as np
import dask.dataframe as dp
from dask import delayed

C:\Users\eddii\AppData\Local\Temp\ipykernel_22404\3686465925.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Dask - lectura de los datos
movies_dask = dp.read_csv('movies.csv')

In [111]:
movies = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

In [112]:
movies = movies.merge(credits, on='title')

In [113]:
movies = movies[['movie_id','title','genres','cast','overview','keywords','crew']]

In [3]:
# isnull with dask?
movies_dask.isnull()

,movieId,title,genres
npartitions=1,,,
,bool,bool,bool
,...,...,...


In [114]:
movies.isnull().sum()

movie_id    0
title       0
genres      0
cast        0
overview    3
keywords    0
crew        0
dtype: int64

In [4]:
# inplace with dask inplace
movies_dask.dropna()

,movieId,title,genres
npartitions=1,,,
,int64,object,object
,...,...,...


In [116]:
movies.dropna(inplace=True)

In [117]:
movies.duplicated().sum()

0

In [27]:
movies_dask.iloc[:,2]

Dask Series Structure:
npartitions=1
    object
       ...
Name: genres, dtype: object
Dask Name: getitem, 2 graph layers


In [118]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [119]:
import ast
ast.literal_eval('[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]')

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [5]:
# funcion con dask
def transform_dask(obj):
    List = []
    for i in obj.split('|'):
        List.append(i)
    return List

In [121]:
def transform(obj):
    List = []
    for i in ast.literal_eval(obj):
        List.append(i['name'])
    return List

In [6]:
# forma de dask para listar los generos:
movies_dask['genres'] = movies_dask['genres'].map(transform_dask)

In [122]:
movies['genres'] = movies['genres'].apply(transform)

In [7]:
movies_dask.head()

,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]


In [123]:
movies.head()

,movie_id,title,genres,cast,overview,keywords,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","In the 22nd century, a paraplegic Marine is di...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","Captain Barbossa, long believed to be dead, ha...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[Action, Adventure, Crime]","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...",A cryptic message from Bond’s past sends him o...,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...",Following the death of District Attorney Harve...,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[Action, Adventure, Science Fiction]","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","John Carter is a war-weary, former military ca...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [26]:
movies['genres'].head()

0    [Action, Adventure, Fantasy, Science Fiction]
1                     [Adventure, Fantasy, Action]
2                       [Action, Adventure, Crime]
3                 [Action, Crime, Drama, Thriller]
4             [Action, Adventure, Science Fiction]
Name: genres, dtype: object

In [124]:
movies['keywords'] = movies['keywords'].apply(transform)

In [125]:
movies['keywords'].head()

0    [culture clash, future, space war, space colon...
1    [ocean, drug abuse, exotic island, east india ...
2    [spy, based on novel, secret agent, sequel, mi...
3    [dc comics, crime fighter, terrorist, secret i...
4    [based on novel, mars, medallion, space travel...
Name: keywords, dtype: object

In [126]:
movies['cast'].isnull().sum()

0

In [128]:
def transform1(obj):
    List = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            List.append(i['name'])
            counter += 1
        else:
            break
    return List

In [129]:
movies['cast'] = movies['cast'].apply(transform1)

In [130]:
movies['cast'].head()

0    [Sam Worthington, Zoe Saldana, Sigourney Weaver]
1       [Johnny Depp, Orlando Bloom, Keira Knightley]
2        [Daniel Craig, Christoph Waltz, Léa Seydoux]
3        [Christian Bale, Michael Caine, Gary Oldman]
4      [Taylor Kitsch, Lynn Collins, Samantha Morton]
Name: cast, dtype: object

In [131]:
def fetch_director(text):
    List = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            List.append(i['name'])
    return List 

In [132]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [133]:
movies['crew'].head()

0        [James Cameron]
1       [Gore Verbinski]
2           [Sam Mendes]
3    [Christopher Nolan]
4       [Andrew Stanton]
Name: crew, dtype: object

In [134]:
movies.head()

,movie_id,title,genres,cast,overview,keywords,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[Sam Worthington, Zoe Saldana, Sigourney Weaver]","In the 22nd century, a paraplegic Marine is di...","[culture clash, future, space war, space colon...",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[Johnny Depp, Orlando Bloom, Keira Knightley]","Captain Barbossa, long believed to be dead, ha...","[ocean, drug abuse, exotic island, east india ...",[Gore Verbinski]
2,206647,Spectre,"[Action, Adventure, Crime]","[Daniel Craig, Christoph Waltz, Léa Seydoux]",A cryptic message from Bond’s past sends him o...,"[spy, based on novel, secret agent, sequel, mi...",[Sam Mendes]
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]","[Christian Bale, Michael Caine, Gary Oldman]",Following the death of District Attorney Harve...,"[dc comics, crime fighter, terrorist, secret i...",[Christopher Nolan]
4,49529,John Carter,"[Action, Adventure, Science Fiction]","[Taylor Kitsch, Lynn Collins, Samantha Morton]","John Carter is a war-weary, former military ca...","[based on novel, mars, medallion, space travel...",[Andrew Stanton]


In [135]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [136]:
movies['overview'].head()

0    [In, the, 22nd, century,, a, paraplegic, Marin...
1    [Captain, Barbossa,, long, believed, to, be, d...
2    [A, cryptic, message, from, Bond’s, past, send...
3    [Following, the, death, of, District, Attorney...
4    [John, Carter, is, a, war-weary,, former, mili...
Name: overview, dtype: object

In [52]:
movies_dask.head()

,movieId,title,genres
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
2,3,Grumpier Old Men (1995),"[Comedy, Romance]"
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
4,5,Father of the Bride Part II (1995),[Comedy]


In [137]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [138]:
movies.head(1)

,movie_id,title,genres,cast,overview,keywords,crew
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction]","[SamWorthington, ZoeSaldana, SigourneyWeaver]","[In, the, 22nd, century,, a, paraplegic, Marin...","[cultureclash, future, spacewar, spacecolony, ...",[JamesCameron]


In [139]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew'] 

In [140]:
df = movies[['movie_id','title','tags']]
df.head()

,movie_id,title,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin..."
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d..."
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send..."
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney..."
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili..."


In [8]:
# Dask separar por espacios
movies_dask['genres'] = movies_dask['genres'].apply(lambda x: " ".join(x))
movies_dask.head()

c:\Python312\Lib\site-packages\dask\dataframe\core.py:4559: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('genres', 'object'))

  warnings.warn(meta_warning(meta))


,movieId,title,genres
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy


In [141]:
df['tags'] = df['tags'].apply(lambda x: " ".join(x))
df.head()

C:\Users\eddii\AppData\Local\Temp\ipykernel_13876\2621718160.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(lambda x: " ".join(x))


,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca..."


In [9]:
# dask lower genres
movies_dask['genres'] = movies_dask['genres'].apply(lambda x:x.lower())
movies_dask.head()

c:\Python312\Lib\site-packages\dask\dataframe\core.py:4559: UserWarning: 
You did not provide metadata, so Dask is running your function on a small dataset to guess output types. It is possible that Dask will guess incorrectly.
To provide an explicit output types or to silence this message, please provide the `meta=` keyword, as described in the map or apply function that you are using.
  Before: .apply(func)
  After:  .apply(func, meta=('genres', 'object'))

  warnings.warn(meta_warning(meta))


,movieId,title,genres
0,1,Toy Story (1995),adventure animation children comedy fantasy
1,2,Jumanji (1995),adventure children fantasy
2,3,Grumpier Old Men (1995),comedy romance
3,4,Waiting to Exhale (1995),comedy drama romance
4,5,Father of the Bride Part II (1995),comedy


In [142]:
df['tags'] = df['tags'].apply(lambda x:x.lower())

C:\Users\eddii\AppData\Local\Temp\ipykernel_13876\841579467.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tags'] = df['tags'].apply(lambda x:x.lower())


In [143]:
df.head()

,movie_id,title,tags
0,19995,Avatar,"in the 22nd century, a paraplegic marine is di..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believed to be dead, ha..."
2,206647,Spectre,a cryptic message from bond’s past sends him o...
3,49026,The Dark Knight Rises,following the death of district attorney harve...
4,49529,John Carter,"john carter is a war-weary, former military ca..."


In [68]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\eddii\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [69]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
from nltk.tokenize import word_tokenize

In [70]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [10]:
# dask vectores genres
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
X = vectorizer.fit_transform(movies_dask['genres'])

In [157]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")
stemmed_feature_names = [stemmer.stem(word) for word in vectorizer.get_feature_names_out()]

In [11]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=10, algorithm='brute', metric='cosine')
knn.fit(X)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=10)

In [144]:
vectors = cv.fit_transform(df['tags'])

In [72]:
feature_names = cv.get_feature_names_out()

In [73]:
stemmed_feature_names = [" ".join([ps.stem(word) for word in word_tokenize(feature)]) for feature in feature_names]

In [74]:
unique_stemmed_feature_names = []
seen = set()

for feature in stemmed_feature_names:
    if feature not in seen:
        unique_stemmed_feature_names.append(feature)
        seen.add(feature)

In [145]:
similarity = cosine_similarity(vectors)

In [12]:
def recommendation_dask(title):
    title_index = movies_dask[movies_dask['title'] == title].index.compute()[0]
    distances, indices = knn.kneighbors(X[title_index])
    recommended_movies_df = movies_dask.compute().iloc[indices[0]]
    recommended_movies = recommended_movies_df['title'].tolist()

    return recommended_movies

In [146]:
def recommend(movie):
    
    movie_lower = movie.lower()
    df['title_lower'] = df['title'].str.lower()
    movie_index1 = df[df['title_lower'] == movie_lower].index

    if not movie_index1.empty:
        movie_index1 = movie_index1[0]
        distances = similarity[movie_index1]
        movies_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:7]
    
        for i in movies_list:
            print(df.iloc[i[0]].title)
    df.drop('title_lower', axis=1, inplace=True)

In [13]:
recommendation_dask("Jumanji (1995)")

['Santa Claus: The Movie (1985)',
 'Golden Compass, The (2007)',
 'NeverEnding Story III, The (1994)',
 'Alice Through the Looking Glass (2016)',
 'Return to Oz (1985)',
 'Chronicles of Narnia: Prince Caspian, The (2008)',
 'Alice in Wonderland (1933)',
 'Bridge to Terabithia (2007)',
 "Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (2001)",
 'Seventh Son (2014)']

In [147]:
recommend('Toy Story')

Toy Story 2
Toy Story 3
The 40 Year Old Virgin
Heartbeeps
Max Keeble's Big Move
Over the Hedge


C:\Users\eddii\AppData\Local\Temp\ipykernel_13876\3950268898.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['title_lower'] = df['title'].str.lower()
C:\Users\eddii\AppData\Local\Temp\ipykernel_13876\3950268898.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop('title_lower', axis=1, inplace=True)
